In [59]:
import pandas as pd
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas
from geopy.geocoders import Nominatim

## Geotagging the scraped Data

In [78]:
df = pd.read_csv('all_reviews.csv')
locations_df = pd.read_csv('locations_geotagged.csv')
beer_df = pd.read_csv('beer_deets.csv')

In [79]:
df

,beer_name,beer_rating,brewery_location,brewery_name,review_content,reviewer_name
0,St. Peter's Ruby Red Ale,3.3,"Bungay, Suffolk",St. Peter's Brewery,"Ruby color, offwhite coam. Aroma and taste: fl...",alex_leit
1,Shepherd Neame Spitfire (Bottle),2.2,"Faversham, Kent",Shepherd Neame,Disappointing show from the lads at ‘Britain’s...,shivermetimber.gray
2,Arbor Rocketman,3.8,"Bristol, Gloucestershire",Arbor Ales,"Puszka. Mętne o żółtym kolorze. Piana biała,ob...",MarcinG
3,Shepherd Neame India Pale Ale (Bottle),3.4,"Faversham, Kent",Shepherd Neame,Bottle. Amber with bronze color and white foam...,alex_leit
4,Shepherd Neame Christmas Ale,3.3,"Faversham, Kent",Shepherd Neame,Bottle. Aroma: English hops and malts with tof...,alex_leit
...,...,...,...,...,...,...
29557,Grain Belt Premium,1.1,-1.5,August Schell Brewing Company,Sampled in a blind taste-test on 11/10/2019. ...,o_Merlin_o
29558,Sierra Nevada Sierraveza,1.2,-1.9,Sierra Nevada Brewing Company,Tasted in a blind taste-test on 11/9/2019. Zer...,o_Merlin_o
29559,Pedavena Birra Dolomiti Pils 4.9,0.5,-2,Fabbrica di Birra di Pedavena (Castello di Udine),"Wygląd: Jasne złoto, przejrzyste bez piany.\n\...",meridian
29560,Steel Reserve Alloy Series Spiked Watermelon 211,4.7,+2,Miller Brewing Company (Molson Coors),Steel Reserve Alloy Series Spiked Watermelon (...,ronaldtheriot


In [62]:
locations_df

,locations,latitude,longitude,country
0,"Bungay, Suffolk",52.456225,1.437341,United Kingdom
1,"Faversham, Kent",51.314409,0.891189,United Kingdom
2,"Bristol, Gloucestershire",51.852767,-2.255369,United Kingdom
3,"Round Rock, Texas",30.508235,-97.678893,United States of America
4,"Brooklyn, New York",40.650104,-73.949582,United States of America
...,...,...,...,...
2376,"Kettering, Ohio",39.689504,-84.168827,United States of America
2377,"Jersey City, New Jersey",40.728158,-74.077642,United States of America
2378,"Hoboken, New Jersey",40.743307,-74.032375,United States of America
2379,"London, Greater London",51.507322,-0.127647,United Kingdom


In [63]:
geolocator = Nominatim(user_agent="None")

In [64]:
locations_new = pd.DataFrame(df.brewery_location[~df.brewery_location.isin(locations_df.locations)].unique(),columns=['locations'])
print(len(locations_new), "new locations to search for! Will take", len(locations_new)/2, "seconds to run.")

19 new locations to search for! Will take 9.5 seconds to run.


In [65]:
def tryconvert(x):
        try:
            return geolocator.geocode(x).latitude
        except:
            return None
    

In [66]:
def tryconvertlong(x):
        try:
            return geolocator.geocode(x).longitude
        except:
            return None

In [67]:
locations_new["latitude"] = locations_new.locations.apply(lambda x: tryconvert(x))

In [68]:
locations_new["longitude"] = locations_new.locations.apply(lambda x: tryconvertlong(x))

In [69]:
countries = []
for row in locations_new.index:
    try:
        loc = geolocator.reverse([locations_new.iloc[row,1],locations_new.iloc[row,2]], language='en')
        countries.append(loc.raw['address']['country'])
    except:
        countries.append(None)
        
    

In [70]:
locations_new['country'] = countries

In [71]:
locations_new

,locations,latitude,longitude,country
0,"West Kelowna, British Columbia",49.857464,-119.580688,Canada
1,"Merritt, British Columbia",50.111704,-120.788423,Canada
2,"Chesham, Buckinghamshire",51.706291,-0.612311,United Kingdom
3,"Wallingford, Connecticut",41.457042,-72.823155,United States of America
4,"Kalona, Iowa",41.483072,-91.706002,United States of America
5,"Laurel, Maryland",39.099275,-76.848306,United States of America
6,"Maria Stein, Ohio",40.407825,-84.493286,United States of America
7,"West Reading, Pennsylvania",40.333704,-75.947432,United States of America
8,"Brantford, Ontario",43.140816,-80.263173,Canada
9,"Berchtesgaden, Bavaria",47.631080,13.002163,Germany


In [72]:
locations_df = pd.concat([locations_df,locations_new])
locations_df.to_csv('locations_geotagged.csv',index=False)

## Merging and cleaning

In [80]:
beer_df

,return_string,flag,type,abv,rb_overall_score,rb_style_score,rb_user_rating,rb_rating_number,name_found
0,St. Peter's Ruby Red Ale\n🇬🇧 Bitter - Ordinary...,🇬🇧,Bitter - Ordinary / Best,4.3,49,95,3.25,415.0,St. Peter's Ruby Red Ale
1,Shepherd Neame Spitfire (Bottle)\n🇬🇧 Bitter - ...,🇬🇧,Bitter - Ordinary / Best,4.5,34,59,3.05,1083.0,Shepherd Neame Spitfire (Bottle)
2,Arbor Rocketman\n🇬🇧 IPA • 6.0%\n95\n97\n3.74\n...,🇬🇧,IPA,6.0,95,97,3.74,59.0,Arbor Rocketman
3,Shepherd Neame India Pale Ale (Bottle)\n🇬🇧 IPA...,🇬🇧,IPA - English,6.1,46,47,3.22,420.0,Shepherd Neame India Pale Ale (Bottle)
4,Shepherd Neame Christmas Ale\n🇬🇧 Strong Ale - ...,🇬🇧,Strong Ale - English,7.0,45,46,3.18,409.0,Shepherd Neame Christmas Ale
...,...,...,...,...,...,...,...,...,...
16442,Truck 59 The Hatch Baptism by Firetruck\n🇨🇦 Ap...,🇨🇦,Apple Cider,NaN,-,-,2.60,1.0,Truck 59 The Hatch Baptism by Firetruck
16443,Barn Owl Breaking Bad Amber Ale\n🇨🇦 Red Ale / ...,🇨🇦,Red Ale / Intl Amber Ale,NaN,-,-,3.30,1.0,Barn Owl Breaking Bad Amber Ale
16444,Yachats / Bailey's Bailey's 10th Anniversary A...,🇺🇸,Belgian Ale - Pale / Golden,0.0,-,-,3.50,6.0,Yachats / Bailey's Bailey's 10th Anniversary Ale
16445,Red Circle Cherry Blaster Gose\n🇨🇦 Gose - Flav...,🇨🇦,Gose - Flavored,0.0,-,-,3.40,2.0,Red Circle Cherry Blaster Gose


In [87]:
# deal with the price issue
price_df = beer_df[beer_df.rb_overall_score.str.contains('Available')]
beer_df = beer_df[~beer_df.rb_overall_score.str.contains('Available')]
price_df['rb_overall_score'] = price_df.return_string.apply(lambda x: x.split('\n')[3])
price_df['rb_style_score'] = price_df.return_string.apply(lambda x: x.split('\n')[4])
price_df['rb_user_rating'] = price_df.return_string.apply(lambda x: x.split('\n')[5])
price_df['rb_rating_number'] = price_df.return_string.apply(lambda x: x.split('\n')[6].replace("(","").replace(")",""))
beer_df = pd.concat([beer_df,price_df])
print("Number of issue beers sorted:", len())


In [81]:
# Creating DF with all details
df_all = pd.merge(df,locations_df,how='left',left_on='brewery_location',right_on='locations')
df_all = pd.merge(df_all, beer_df, how='left', left_on='beer_name',right_on='name_found')
df_all


,beer_name,beer_rating,brewery_location,brewery_name,review_content,reviewer_name,locations,latitude,longitude,country,return_string,flag,type,abv,rb_overall_score,rb_style_score,rb_user_rating,rb_rating_number,name_found
0,St. Peter's Ruby Red Ale,3.3,"Bungay, Suffolk",St. Peter's Brewery,"Ruby color, offwhite coam. Aroma and taste: fl...",alex_leit,"Bungay, Suffolk",52.456225,1.437341,United Kingdom,St. Peter's Ruby Red Ale\n🇬🇧 Bitter - Ordinary...,🇬🇧,Bitter - Ordinary / Best,4.3,49,95,3.25,415.0,St. Peter's Ruby Red Ale
1,Shepherd Neame Spitfire (Bottle),2.2,"Faversham, Kent",Shepherd Neame,Disappointing show from the lads at ‘Britain’s...,shivermetimber.gray,"Faversham, Kent",51.314409,0.891189,United Kingdom,Shepherd Neame Spitfire (Bottle)\n🇬🇧 Bitter - ...,🇬🇧,Bitter - Ordinary / Best,4.5,34,59,3.05,1083.0,Shepherd Neame Spitfire (Bottle)
2,Arbor Rocketman,3.8,"Bristol, Gloucestershire",Arbor Ales,"Puszka. Mętne o żółtym kolorze. Piana biała,ob...",MarcinG,"Bristol, Gloucestershire",51.852767,-2.255369,United Kingdom,Arbor Rocketman\n🇬🇧 IPA • 6.0%\n95\n97\n3.74\n...,🇬🇧,IPA,6.0,95,97,3.74,59.0,Arbor Rocketman
3,Shepherd Neame India Pale Ale (Bottle),3.4,"Faversham, Kent",Shepherd Neame,Bottle. Amber with bronze color and white foam...,alex_leit,"Faversham, Kent",51.314409,0.891189,United Kingdom,Shepherd Neame India Pale Ale (Bottle)\n🇬🇧 IPA...,🇬🇧,IPA - English,6.1,46,47,3.22,420.0,Shepherd Neame India Pale Ale (Bottle)
4,Shepherd Neame Christmas Ale,3.3,"Faversham, Kent",Shepherd Neame,Bottle. Aroma: English hops and malts with tof...,alex_leit,"Faversham, Kent",51.314409,0.891189,United Kingdom,Shepherd Neame Christmas Ale\n🇬🇧 Strong Ale - ...,🇬🇧,Strong Ale - English,7.0,45,46,3.18,409.0,Shepherd Neame Christmas Ale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29840,Grain Belt Premium,1.1,-1.5,August Schell Brewing Company,Sampled in a blind taste-test on 11/10/2019. ...,o_Merlin_o,-1.5,52.583680,13.276599,Germany,Grain Belt Premium\n🇺🇸 Pale Lager - North Am. ...,🇺🇸,Pale Lager - North Am. / Light / Rice,4.6,11,91,2.84,285.0,Grain Belt Premium
29841,Sierra Nevada Sierraveza,1.2,-1.9,Sierra Nevada Brewing Company,Tasted in a blind taste-test on 11/9/2019. Zer...,o_Merlin_o,-1.9,49.024169,9.028337,Germany,Sierra Nevada Sierraveza\n🇺🇸 Pale Lager - Nort...,🇺🇸,Pale Lager - North Am. / Light / Rice,5.0,42,99,3.18,119.0,Sierra Nevada Sierraveza
29842,Pedavena Birra Dolomiti Pils 4.9,0.5,-2,Fabbrica di Birra di Pedavena (Castello di Udine),"Wygląd: Jasne złoto, przejrzyste bez piany.\n\...",meridian,-2,-23.604131,-69.084278,Chile,Pedavena Birra Dolomiti Pils 4.9\n🇮🇹 Pale Lage...,🇮🇹,Pale Lager - North Am. / Light / Rice,4.9,8,85,2.52,156.0,Pedavena Birra Dolomiti Pils 4.9
29843,Steel Reserve Alloy Series Spiked Watermelon 211,4.7,+2,Miller Brewing Company (Molson Coors),Steel Reserve Alloy Series Spiked Watermelon (...,ronaldtheriot,+2,-23.604131,NaN,NaN,Steel Reserve Alloy Series Spiked Watermelon 2...,🇺🇸,Malt Liquor,8.0,13,91,2.86,33.0,Steel Reserve Alloy Series Spiked Watermelon 211


In [82]:
# drop excess columns
df_all.drop(["locations","return_string","name_found"],axis=1,inplace=True)
# drop rows with ratings instead of location
df_all = df_all[~df_all['brewery_location'].str.len()<4]
# drop all NaNs
print(len(df_all) - len(df_all.dropna()), "columns with NaNs dropped.")
df_all.dropna(inplace=True)


2329 columns with NaNs dropped.


In [83]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27516 entries, 0 to 29844
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   beer_name         27516 non-null  object 
 1   beer_rating       27516 non-null  float64
 2   brewery_location  27516 non-null  object 
 3   brewery_name      27516 non-null  object 
 4   review_content    27516 non-null  object 
 5   reviewer_name     27516 non-null  object 
 6   latitude          27516 non-null  float64
 7   longitude         27516 non-null  float64
 8   country           27516 non-null  object 
 9   flag              27516 non-null  object 
 10  type              27516 non-null  object 
 11  abv               27516 non-null  float64
 12  rb_overall_score  27516 non-null  object 
 13  rb_style_score    27516 non-null  object 
 14  rb_user_rating    27516 non-null  object 
 15  rb_rating_number  27516 non-null  float64
dtypes: float64(5), object(11)
memory usage: 

## Deal with Non-English Descriptions

## Save down cleaned data

In [77]:
df_all.to_csv("data_clean.csv",index=False)